In [3]:
!rm -rf Eunomia
!git clone https://github.com/ouaguenouni/Eunomia
!pip install -r Eunomia/requirements.txt

Clonage dans 'Eunomia'...
remote: Enumerating objects: 429, done.
remote: Counting objects: 100% (429/429), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 429 (delta 333), reused 386 (delta 290), pack-reused 0
Réception d'objets: 100% (429/429), 2.63 Mio | 4.29 Mio/s, fait.
Résolution des deltas: 100% (333/333), fait.


In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from Eunomia.preferences import *
from Eunomia.additive_functions import *
from Eunomia.alternatives import *
from Eunomia.sampling import *
from Eunomia.mcmc import *
#from Eunomia.degree import *
from Eunomia.experiments import *
import torch
import torch.nn as nn

pyro.set_rng_seed(1)

In [134]:
class PhiFunction(nn.Module):
    def __init__(self, theta):
        super(PhiFunction, self).__init__()
        n = len(theta)
        self.theta = theta
        self.phi_values = {}
        for i_A in range(len(self.theta)):
            A = theta[i_A]
            for i_B in range(i_A + 1, len(self.theta)):
                B = theta[i_B]
                if i_A == i_B:
                    continue
                for X in [0,1]:
                    for Y in [0,1]:
                        self.phi_values[tuple(A),tuple(B),X,Y] = nn.Parameter(torch.randn(1))
        

    def forward(self, sparse_X, sparse_Y):
        phi_sum = 0
        for i_A in range(len(self.theta)):
            A = theta[i_A]
            for i_B in range(i_A + 1, len(self.theta)):
                B = theta[i_B]
                if i_A == i_B:
                    continue
                v = self.phi_values[tuple(A),
                                    tuple(B),
                                    int(all(i in sparse_X for i in A)),
                                    int(all(i in sparse_Y for i in B))
                                    ]
                phi_sum += v
        return phi_sum[0]
    
    def prior(self):
        phis = [torch.abs(self.phi_values[k]) for k in self.phi_values]
        return sum(phis)
    

In [157]:
class SSBModel(nn.Module):
    def __init__(self, theta):
        super(SSBModel, self).__init__()
        self.theta = theta
        self.weights = nn.Parameter(torch.randn(len(theta)))
        self.phi_function = PhiFunction(theta)
        
    def reg_term(self, alpha, beta):
        return alpha*torch.sum(torch.pow(self.weights, torch.tensor(2))) + beta*self.phi_function.prior()
    
    def likelihood(self, R):
        l = 0
        for sparse_X,sparse_Y in R:
            f = self.forward(sparse_X, sparse_Y)
            l += torch.sigmoid(f)
        return l

    def forward(self, sparse_X, sparse_Y):
        f = 0
        for S in self.theta:
            if all(i in sparse_X for i in S):
                f += self.weights[self.theta.index(S)]
            if all(i in sparse_Y for i in S):
                f -= self.weights[self.theta.index(S)]
        f += self.phi_function(sparse_X, sparse_X)
        return f

In [158]:
an = 3
m = 2**(n-1)
k = 1
sigma_weights = 10

theta = generate_additive_theta(n, k)
weights = generate_normal_weights(theta, sigma_weights)
alternatives = generate_random_alternatives_matrix(m, n)
ranks = compute_ws_ranks(alternatives, theta, weights)
t_sv = compute_semivalues(n, theta, weights, lambda x:1)
preferences = PreferenceModel(alternatives, ranks)
s = preferences.generate_sparse_preference_set()
print(s)

{((), (0, 2)), ((1, 2), (2,)), ((), (2,)), ((), (1, 2)), ((0, 2), (2,)), ((0, 2), (1, 2))}


In [164]:
model = SSBModel(theta)
print("w coefficients = ", model.weights)
for i,j,A,B in model.phi_function.phi_values: 
    print("\n ==============")
    print(f"C[{i}, {j}, {A}, {B}] = {model.phi_function.phi_values[i,j,A,B]}", end = " \n")
    print("\n ==============")


w coefficients =  Parameter containing:
tensor([-1.0025, -0.0356, -1.3917], requires_grad=True)

C[(0,), (1,), 0, 0] = Parameter containing:
tensor([-0.3523], requires_grad=True) 


C[(0,), (1,), 0, 1] = Parameter containing:
tensor([1.9390], requires_grad=True) 


C[(0,), (1,), 1, 0] = Parameter containing:
tensor([0.4224], requires_grad=True) 


C[(0,), (1,), 1, 1] = Parameter containing:
tensor([0.0572], requires_grad=True) 


C[(0,), (2,), 0, 0] = Parameter containing:
tensor([-2.1116], requires_grad=True) 


C[(0,), (2,), 0, 1] = Parameter containing:
tensor([2.1080], requires_grad=True) 


C[(0,), (2,), 1, 0] = Parameter containing:
tensor([0.3232], requires_grad=True) 


C[(0,), (2,), 1, 1] = Parameter containing:
tensor([-0.3847], requires_grad=True) 


C[(1,), (2,), 0, 0] = Parameter containing:
tensor([-0.2699], requires_grad=True) 


C[(1,), (2,), 0, 1] = Parameter containing:
tensor([0.4822], requires_grad=True) 


C[(1,), (2,), 1, 0] = Parameter containing:
tensor([-0.3718

In [181]:
import torch.optim as optim

model = SSBModel(theta)

optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 2000  # Define the number of epochs
for epoch in range(num_epochs):
    total_loss = 0
    optimizer.zero_grad()
    # Compute likelihood and prior
    likelihood = model.likelihood(s)
    # Loss is negative log-likelihood minus log-prior (for maximizing posterior)
    loss = -torch.log(likelihood) + model.reg_term(1,1)
    # Backpropagation
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    if epoch % 50 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss}")

Epoch 1/2000, Loss: 10.960253715515137
Epoch 51/2000, Loss: 10.768819808959961
Epoch 101/2000, Loss: 10.596577644348145
Epoch 151/2000, Loss: 10.441316604614258
Epoch 201/2000, Loss: 10.300185203552246
Epoch 251/2000, Loss: 10.170719146728516
Epoch 301/2000, Loss: 10.050957679748535
Epoch 351/2000, Loss: 9.939445495605469
Epoch 401/2000, Loss: 9.835147857666016
Epoch 451/2000, Loss: 9.737339973449707
Epoch 501/2000, Loss: 9.645502090454102
Epoch 551/2000, Loss: 9.559248924255371
Epoch 601/2000, Loss: 9.478282928466797
Epoch 651/2000, Loss: 9.402347564697266
Epoch 701/2000, Loss: 9.331218719482422
Epoch 751/2000, Loss: 9.264684677124023
Epoch 801/2000, Loss: 9.202545166015625
Epoch 851/2000, Loss: 9.14460277557373
Epoch 901/2000, Loss: 9.090668678283691
Epoch 951/2000, Loss: 9.040555000305176
Epoch 1001/2000, Loss: 8.994077682495117
Epoch 1051/2000, Loss: 8.951055526733398
Epoch 1101/2000, Loss: 8.91131591796875
Epoch 1151/2000, Loss: 8.874683380126953
Epoch 1201/2000, Loss: 8.840990066

In [182]:
s

{((), (0, 2)),
 ((), (1, 2)),
 ((), (2,)),
 ((0, 2), (1, 2)),
 ((0, 2), (2,)),
 ((1, 2), (2,))}

In [183]:
model = SSBModel(theta)
print("w coefficients = ", model.weights)
for i,j,A,B in model.phi_function.phi_values: 
    print("\n ==============")
    print(f"C[{i}, {j}, {A}, {B}] = {model.phi_function.phi_values[i,j,A,B]}", end = " \n")
    print("\n ==============")

w coefficients =  Parameter containing:
tensor([ 1.4955,  0.7525, -0.3977], requires_grad=True)

C[(0,), (1,), 0, 0] = Parameter containing:
tensor([-1.2727], requires_grad=True) 


C[(0,), (1,), 0, 1] = Parameter containing:
tensor([-0.1921], requires_grad=True) 


C[(0,), (1,), 1, 0] = Parameter containing:
tensor([0.6353], requires_grad=True) 


C[(0,), (1,), 1, 1] = Parameter containing:
tensor([-1.3944], requires_grad=True) 


C[(0,), (2,), 0, 0] = Parameter containing:
tensor([-0.9095], requires_grad=True) 


C[(0,), (2,), 0, 1] = Parameter containing:
tensor([0.1111], requires_grad=True) 


C[(0,), (2,), 1, 0] = Parameter containing:
tensor([-0.0352], requires_grad=True) 


C[(0,), (2,), 1, 1] = Parameter containing:
tensor([-0.8154], requires_grad=True) 


C[(1,), (2,), 0, 0] = Parameter containing:
tensor([0.2774], requires_grad=True) 


C[(1,), (2,), 0, 1] = Parameter containing:
tensor([0.6695], requires_grad=True) 


C[(1,), (2,), 1, 0] = Parameter containing:
tensor([0.986